# 7 Download rtl results from s3 and merges results for all dataset

**Execute locally**

Requires:
* dataset/final_results.csv for every dataset name in LOCAL_DATASETS
* dataset/final_results.csv for every dataset in S3_DATASETS

Returns:
* final_results.csv containing the results contained in the input files with an additional column denoting the dataset

In [ ]:
LOCAL_DATASETS = ["lastfm_10_pc", "ml-10m"]
S3_DATASETS = ["rtl"]

In [ ]:
import os
import subprocess

from io import StringIO

import boto3
import pandas as pd

In [ ]:
def download_s3_csv(bucket, key):
    client = boto3.client('s3')
    csv_obj = client.get_object(Bucket=bucket, Key=key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    buffer = StringIO(csv_string)
    return buffer

In [ ]:
local_data_path = "/Users/nknyazev/Documents/Delft/Thesis/temporal/data/results/RQ3"
s3_bucket = "ci-data-apps"
s3_folder_key = "norman/sagemaker/thesis/offline-evaluation/RQ3"
data_filename = 'final_results.csv'

output_path = "/Users/nknyazev/Documents/Delft/Thesis/temporal/data/results/RQ3/final_results.csv"

In [ ]:
for d in LOCAL_DATASETS:
    _ = subprocess.call(["scp", f"nknyazev@student-linux.tudelft.nl:/tudelft.net/staff-bulk/ewi/insy/MMC/nknyazev/RQ3/{d}/{data_filename}", os.path.join(local_data_path, d, data_filename)])

In [ ]:
local_dfs = {d: pd.read_csv(os.path.join(local_data_path, d, data_filename), sep='\t') for d in LOCAL_DATASETS}
s3_dfs = {d: pd.read_csv(download_s3_csv(s3_bucket, os.path.join(s3_folder_key, d, data_filename)), sep='\t') for d in S3_DATASETS}

In [ ]:
dfs = {**local_dfs, **s3_dfs}

In [ ]:
for d in dfs:
    original_columns = dfs[d].columns.tolist()
    dfs[d]['dataset'] = d
    dfs[d] = dfs[d][['dataset'] + original_columns]

In [ ]:
df = pd.concat(list(dfs.values()), ignore_index=True)

In [ ]:
df.to_csv(output_path, sep="\t", index=False)